In [1]:
"""
Explorar Dataset RSNA
[O Dataset chest-xRay-14 não é ideal para estudar pneumonia, ele não possui imagens o suficiente]
1. Identificar Problemas no Dataset;
2. Caso existirem problemas, resolvê-los;
3. Identificar quantas imagens existem com pneumonia e sem pneumonia;
4. Bora Codar, RetinaNet e DenseNet
5. Gerar Heatmap
...
...
...
6. Pensar em como fazer o fine-tunning

"""
import pandas as pd
import os

#As imagens de TEST não possuem Labels
#Existem 26684 imagens na pasta de treinamento, contagem diferente do .CSV
#Isso ocorre pois algumas imagens apresentam 2 ou mais pontos de ofuscamento/pneumonia

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [3]:
##############################################
###########  ORIGINAL DATAFRAME  #############
##############################################

df_train = pd.read_csv("rsna-pneumonia-detection-challenge/stage_2_train_labels.csv")

In [4]:
df_train.head()
#NaN Indicam que os campos não existem (,,,,)
#Olhar sempre ao campo Target primeiro

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [5]:
df_train.info()
#PatientID -> ID unico representando cada imagem
#x,y -> Cordenadas superior esquerda da localidade da pneumonia
#width, height -> Tamanho do Bounding Box
#Target -> 0 não possui pneumonia, 1 possui pneumonia

#O csv contém 30227 entradas, mas só existem 26684 imagens

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30227 entries, 0 to 30226
Data columns (total 6 columns):
patientId    30227 non-null object
x            9555 non-null float64
y            9555 non-null float64
width        9555 non-null float64
height       9555 non-null float64
Target       30227 non-null int64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.4+ MB


In [6]:
com_pneumonia = len(df_train[df_train['Target'] == 1])
sem_pneumonia = len(df_train[df_train['Target'] == 0])
print(".:: CSV ORIGINAL ::.")
print("Total de {} entradas".format(com_pneumonia + sem_pneumonia))
print("{} entradas que contem pneumonia (podem ser mais de 1 para uma mesma imagem)".format(com_pneumonia))
print("{} entradas que não contem pneumonia".format(sem_pneumonia))
print("Relação de {:.2f} entradas com pneumonia para uma entrada sem pneumonia".format(com_pneumonia/sem_pneumonia))

.:: CSV ORIGINAL ::.
Total de 30227 entradas
9555 entradas que contem pneumonia
20672 entradas que não contem pneumonia
Relação de 0.46 entradas com pneumonia para uma entrada sem pneumonia


In [7]:
##############################################
#############  NEW DATAFRAME  ################
##############################################

patient_ids = [os.path.splitext(f)[0] for f in listdir_nohidden("rsna-pneumonia-detection-challenge/stage_2_train_images/")]
print("Existem {} imagens para treinamento".format(len(patient_ids)))


Existem 26684 imagens para treinamento


In [8]:
#Criaremos um novo CSV incluindo SOMENTE os IDS UNICOS que existem no diretório de treinamento
df_training_images_ids = pd.DataFrame(columns=df_train.columns)
i = 0
for id in patient_ids:
    index = df_train.loc[df_train['patientId'] == id].index.values.astype(int)[0]
    df_training_images_ids.loc[i] = df_train.iloc[index]
    i = i + 1


In [10]:
print("Quantidade de imagens com Pneumonia (Target=1)")
df_training_images_ids[df_training_images_ids['Target'] == 1]

Quantidade de imagens com Pneumonia (Target=1)


,patientId,x,y,width,height,Target
3,22f2d3ec-f7ea-4778-850d-bb111590202f,213.0,321.0,169.0,137.0,1
7,1caa4dac-4bac-419b-91d4-cac2d8408ccd,318.0,319.0,156.0,267.0,1
10,ebc9da35-fa1b-4cc4-b963-16731802af49,342.0,496.0,138.0,221.0,1
20,11a2e0cb-ee1f-4d53-a211-b8faff6bf888,617.0,608.0,247.0,130.0,1
21,505e3473-fe7f-4c3e-8684-b7f9ecf1d052,524.0,272.0,181.0,258.0,1
22,ac710523-7dfb-4d63-821c-34b2d6ef03d7,687.0,607.0,178.0,134.0,1
23,3a483825-5dc2-4318-b0a6-7378f155df2c,542.0,174.0,354.0,606.0,1
25,af59f4a2-32d4-4888-ac7f-fc0bcc23abd2,315.0,571.0,167.0,254.0,1
27,c5e9b1c8-4a98-4ea6-b918-b1621ecf0ddd,569.0,245.0,216.0,441.0,1
38,32c3e235-1885-4aad-b4ad-2ef8583fbb78,604.0,676.0,193.0,153.0,1


In [11]:
print("Quantidade de imagens SEM Pneumonia (Target=0)")
df_training_images_ids[df_training_images_ids['Target'] == 0]

Quantidade de imagens SEM Pneumonia (Target=0)


,patientId,x,y,width,height,Target
0,7be6b4de-afe9-43c0-a581-0f49608c8976,NaN,NaN,NaN,NaN,0
1,2dcdd159-2889-48d3-a0ce-5c7b1086c49d,NaN,NaN,NaN,NaN,0
2,d8e66874-305e-4c80-9b75-5e764eb718ff,NaN,NaN,NaN,NaN,0
4,cdaa07d4-4234-4cd2-b9bf-abbf5aed1bb4,NaN,NaN,NaN,NaN,0
5,46c4f908-9292-437f-af42-4031fca621f2,NaN,NaN,NaN,NaN,0
6,79fa156c-c4f5-478d-a14d-80cc4db5cb8d,NaN,NaN,NaN,NaN,0
8,e8a6ccf4-845e-4663-b561-008bdf13c7fd,NaN,NaN,NaN,NaN,0
9,6a975635-1118-4c26-b613-43fd34ce8b16,NaN,NaN,NaN,NaN,0
11,ce30917f-c3bf-47fa-ab69-a1062f387f3a,NaN,NaN,NaN,NaN,0
12,79cb4053-b062-4be9-8922-9585b170d8fc,NaN,NaN,NaN,NaN,0


In [12]:
#Relacionando as quantidades de imagens
com_pneumonia = len(df_training_images_ids[df_training_images_ids['Target'] == 1])
sem_pneumonia = len(df_training_images_ids[df_training_images_ids['Target'] == 0])
print("Total de {} imagens".format(com_pneumonia + sem_pneumonia))
print("{} imagens com pneumonia".format(com_pneumonia))
print("{} imagens sem pneumonia".format(sem_pneumonia))
print("Relação de {:.2f} imagem com pneumonia para uma imagem sem pneumonia".format(com_pneumonia/sem_pneumonia))


Total de 26684 imagens
6012 imagens com pneumonia
20672 imagens sem pneumonia
Relação de 0.29 imagem com pneumonia para uma imagem sem pneumonia


In [13]:
#Exportando o novo Dataframe
#Esse dataframe contem os IDS dos pacientes para todas as 26684 imagens existentes nos diretórios de treinamento, são eles:
#stage_2_train_images (Formato DICOM) e stage_2_train_images_png (Formato PNG)
df_training_images_ids = df_training_images_ids[df_training_images_ids.columns[0]]
df_training_images_ids.to_csv('rsna-pneumonia-detection-challenge/ids_pacientes_treinamento.csv', index=False)